In [1]:
#basic imports
from langchain_core.tools import tool,Tool
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

import os

# special imports
from langchain_community.tools import DuckDuckGoSearchRun ###simple search tool
from langchain_community.tools import DuckDuckGoSearchResults ### returns snipeet,title,link
from langchain_community.utilities import GoogleSerperAPIWrapper
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
from io import BytesIO
import requests
from langchain_experimental.utilities import PythonREPL


load_dotenv() 

True

In [ ]:
# --- Basic operations --- #

@tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers.
    Args:
        a (float): the first number
        b (float): the second number
    """
    return a * b


@tool
def add(a: float, b: float) -> float:
    """Adds two numbers.
    Args:
        a (float): the first number
        b (float): the second number
    """
    return a + b


@tool
def subtract(a: float, b: float) -> int:
    """Subtracts two numbers.
    Args:
        a (float): the first number
        b (float): the second number
    """
    return a - b


@tool
def divide(a: float, b: float) -> float:
    """Divides two numbers.
    Args:
        a (float): the first float number
        b (float): the second float number
    """
    if b == 0:
        raise ValueError("Cannot divided by zero.")
    return a / b


@tool
def modulus(a: int, b: int) -> int:
    """Get the modulus of two numbers.
    Args:
        a (int): the first number
        b (int): the second number
    """
    return a % b


@tool
def power(a: float, b: float) -> float:
    """Get the power of two numbers.
    Args:
        a (float): the first number
        b (float): the second number
    """
    return a**b


@tool
def largest(a:list[float]) -> float;
    """Get the largest number in a list.
    Args:
        a (list[float]): the list of numbers
    """
    return max(a)

@tool
def smallest(a:list[float]) -> float;
    """Get the smallest number in a list.
    Args:
        a (list[float]): the list of numbers
    """
    return min(a)
@tool
def average(a:list[float]) -> float;
    """Get the average of a list of numbers.
    Args:
        a (list[float]): the list of numbers
    """
    return sum(a)/len(a)




In [2]:
# --- Advanced operations --- #

# duckduckgo search tool
simple_search = DuckDuckGoSearchRun()


# duckduckgo search tool returns snipeet,title,link
search = DuckDuckGoSearchResults(output_format="list")


# google search tool 
search = GoogleSerperAPIWrapper()
# Need to explicitly convert search function to a tool
google_search = Tool(
    name="Google Search",
    func=lambda q: str(search.results(q)),  # returns list of dicts with 'link'
    description="Search Google and get structured results including links.",
)


#python interpreter
python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

In [3]:
repl_tool.invoke("print(1+1)")

Python REPL can execute arbitrary code. Use with caution.


'2\n'

In [ ]:


def get_webpage_content(url: str) -> str:
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        content_type = response.headers.get("Content-Type", "").lower()

        # Check if it's a PDF
        if "application/pdf" in content_type or url.lower().endswith(".pdf"):
            pdf = PdfReader(BytesIO(response.content))
            text = ""
            for page in pdf.pages:
                text += page.extract_text() or ""
            return text[:2000]  # Truncate for safety

        # Else assume it's HTML
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator="\n", strip=True)
        return text[:2000]

    except Exception as e:
        return f"Error fetching content: {e}"
    
webpage_tool = Tool(
    name="get_webpage_content",
    func=get_webpage_content,
    description="Fetches and returns main text from a webpage or a PDF URL."
)

In [33]:
chat=ChatOpenAI()

In [36]:
chat.invoke("What is the capital of India?")

parser=StrOutputParser()

parser.invoke(chat.invoke("What is the capital of India?"))



'The capital of India is New Delhi.'

In [17]:
from toolkit import tool_list

tool_list[10]

Tool(name='DuckDuckGo Search', description="Search the web using DuckDuckGo and return a list of relevant results. Useful for answering general knowledge questions or finding recent news.The output is a list of dictionaries, each containing 'title', 'snippet', and 'link' keys.The 'link' key contains the URL of the result, which can be used to fetch more information.", func=<bound method BaseTool.run of DuckDuckGoSearchResults(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'), output_format='list')>)

In [19]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia_search=Tool(
    name="wikipedia_search",
    func=wikipedia.run,
    description=(
        "search argument should be at least 3 words long. "
        "Fetches summaries from Wikipedia. Useful for general knowledge or factual questions about entities, events, concepts, etc."
    "Prioritze this wikipedia search tool over web search and get web content for general knowledge questions. "
    )

)


In [26]:
wikipedia_search.invoke("mily Midkiff June 2014 article Hreidmar's sons journal dragon depiction")

'No good Wikipedia Search Result was found'

In [46]:
# Google search tool
search = GoogleSerperAPIWrapper()

# Need to explicitly convert search function to a tool
google_search_tool = Tool(
    name="Google_Search",
    func=lambda q: search.results(q)['organic'], # Limit to top 3 results
    description="Search web using Google and get structured results that includes links, which can be used to fetch more information.",
)

In [ ]:
google_search_tool.invoke("Emily Midkiff June 2014 article Hreidmar's sons journal dragon depiction")

[{'title': '[PDF] Adaptive In-conversation Team Building for Language Model Agents',
  'link': 'https://arxiv.org/pdf/2405.19425?',
  'snippet': "Identify the journal named for one of Hreidmar's sons that guarded his house. 2. Locate Emily Midkiff's June 2014 article in that journal. 3 ...",
  'date': 'Mar 2, 2025',
  'attributes': {'Missing': 'mily | Show results with:mily'},
  'position': 1},
 {'title': "“Dragons are Tricksy”: The Uncanny Dragons of Children's Literature |",
  'link': 'https://journal.finfar.org/articles/dragons-are-tricksy-the-uncanny-dragons-of-childrens-literature/',
  'snippet': "This article argues that children's literature dragons have been not been entirely softened and satirized.",
  'attributes': {'Missing': "Hreidmar's | Show results with:Hreidmar's"},
  'position': 2},
 {'title': 'm-ric/agents_small_benchmark · Datasets at Hugging Face',
  'link': 'https://huggingface.co/datasets/m-ric/agents_small_benchmark',
  'snippet': "In Emily Midkiff's June 2014 ar

In [48]:
os.getenv("SPACE_ID")

In [49]:
print(os.getenv("SPACE_ID"))

None


In [53]:
from toolkit import tool_list

In [59]:
print(tool_list[:])

[StructuredTool(name='multiply', description='Multiplies two numbers.\n    Args:\n        a (float): the first number\n        b (float): the second number', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x115fae9e0>), StructuredTool(name='add', description='Adds two numbers.\n    Args:\n        a (float): the first number\n        b (float): the second number', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x116157eb0>), StructuredTool(name='subtract', description='Subtracts two numbers.\n    Args:\n        a (float): the first number\n        b (float): the second number', args_schema=<class 'langchain_core.utils.pydantic.subtract'>, func=<function subtract at 0x116157a30>), StructuredTool(name='divide', description='Divides two numbers.\n    Args:\n        a (float): the first float number\n        b (float): the second float number', args_schema=<class 'langchain_core.utils.pydantic.divide'>, func=<function

In [57]:
print(i for i in tool_list)

<generator object <genexpr> at 0x124c4b0d0>


In [60]:
from pytube import extract

ModuleNotFoundError: No module named 'pytube'